# Beat Reflection Processing Tester

This notebook allows you to test the JSON parsing and relationship update functions with raw LLM responses.

In [3]:
# Setup and imports
import sys
import os
import json

# Add the current directory to Python path
sys.path.insert(0, '.')

from prompts.llm_interface import LLMInterface
from npc.npc_manager import NPCManager

# Initialize the LLM interface
llm = LLMInterface()

print("✅ Setup complete!")
print("You can now test JSON parsing and relationship updates.")

✅ Setup complete!
You can now test JSON parsing and relationship updates.


## Test JSON Parsing

Paste your raw JSON response in the cell below and run it to test the parsing.

In [5]:
# PASTE YOUR RAW RESPONSE HERE
# You can include the ```json markers or just the JSON
raw_response = '''
```json
{
  "relationships": {
    "Alice": {
      "label": "Cautious Observer",
      "trust_delta": -1,
      "affection_delta": 0,
      "beat_memory": "Her persistent unease, while frustrating, highlighted a crucial difference in our approaches to problem-solving."
    },
    "Bob": {
      "label": "Pragmatic Skeptic",
      "trust_delta": -2,
      "affection_delta": -1,
      "beat_memory": "His dismissive attitude towards her concerns reinforced my initial feeling that he prioritized practical solutions over deeper investigation.”
    }
  },
  "knowledge_gained": {
    "facts": [
      "The storage room was forcibly entered.",
      "Someone stole our food.",
      "Bob is focused on securing the physical evidence and minimizing potential 'fantastical scenarios'."
    ],
    "suspicions": [
      "Bob's insistence on dismissing Alice's concerns suggests he might be deliberately overlooking alternative explanations.",
      "The forced entry indicates a deliberate act, potentially involving someone who knew about our food supply."
    ],
    "gossip_worthy": [
      "Bob's aggressive examination of the lock could be a tactic to intimidate or mislead.",
      "Alice's intuitive feelings might be a valuable asset, despite Bob's disapproval."
    ]
  }
}
```
'''

In [40]:
import re
json_text = raw_response

if "```json" in json_text:
    # Extract content between ```json and ```
    start = json_text.find("```json") + 7
    end = json_text.find("```", start)
    if end != -1:
        json_text = json_text[start:end].strip()
        
json_text = json_text.replace(""", '"').replace(""", '"').replace("'", "'").replace("'", "'")
            # Also handle smart/curly apostrophes and additional quote variants
json_text = json_text.replace("'", "'").replace("'", "'").replace("‚", "'").replace("„", '"').replace("‟", '"').replace("”",'"')

json_text = re.sub(r'[\r\n\t]', ' ', json_text)
# Remove other control characters that can break JSON parsing
json_text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', json_text)
# Clean up multiple spaces
json_text = re.sub(r'\s+', ' ', json_text)

print("First 200 characters:")
print(repr(json_text[:200]))
print("\nFull text:")
print(json_text)

json.loads(json_text)



First 200 characters:
'{ "relationships": { "Alice": { "label": "Cautious Observer", "trust_delta": -1, "affection_delta": 0, "beat_memory": "Her persistent unease, while frustrating, highlighted a crucial difference in our'

Full text:
{ "relationships": { "Alice": { "label": "Cautious Observer", "trust_delta": -1, "affection_delta": 0, "beat_memory": "Her persistent unease, while frustrating, highlighted a crucial difference in our approaches to problem-solving." }, "Bob": { "label": "Pragmatic Skeptic", "trust_delta": -2, "affection_delta": -1, "beat_memory": "His dismissive attitude towards her concerns reinforced my initial feeling that he prioritized practical solutions over deeper investigation." } }, "knowledge_gained": { "facts": [ "The storage room was forcibly entered.", "Someone stole our food.", "Bob is focused on securing the physical evidence and minimizing potential 'fantastical scenarios'." ], "suspicions": [ "Bob's insistence on dismissing Alice's concerns suggests he

{'relationships': {'Alice': {'label': 'Cautious Observer',
   'trust_delta': -1,
   'affection_delta': 0,
   'beat_memory': 'Her persistent unease, while frustrating, highlighted a crucial difference in our approaches to problem-solving.'},
  'Bob': {'label': 'Pragmatic Skeptic',
   'trust_delta': -2,
   'affection_delta': -1,
   'beat_memory': 'His dismissive attitude towards her concerns reinforced my initial feeling that he prioritized practical solutions over deeper investigation.'}},
 'knowledge_gained': {'facts': ['The storage room was forcibly entered.',
   'Someone stole our food.',
   "Bob is focused on securing the physical evidence and minimizing potential 'fantastical scenarios'."],
  'suspicions': ["Bob's insistence on dismissing Alice's concerns suggests he might be deliberately overlooking alternative explanations.",
   'The forced entry indicates a deliberate act, potentially involving someone who knew about our food supply.'],
  'gossip_worthy': ["Bob's aggressive exam

In [9]:
import re
import json
from typing import Dict, Any

def _parse_json_response(raw_response):
    """Unified JSON parsing function (same as mafia.py)"""
    try:
        # Extract JSON from markdown code blocks if present
        json_text = raw_response
        print(json_text)
        if "```json" in json_text:
            # Extract content between ```json and ```
            start = json_text.find("```json") + 7
            end = json_text.find("```", start)
            if end != -1:
                json_text = json_text[start:end].strip()
        
        # Fix all Unicode quotes that break JSON parsing (exactly like mafia.py)
        json_text = json_text.replace(""", '"').replace(""", '"').replace("'", "'").replace("'", "'")
        # Also handle smart/curly apostrophes and additional quote variants
        json_text = json_text.replace("'", "'").replace("'", "'").replace("‚", "'").replace("„", '"').replace("‟", '"').replace("”",'"')
        
        # More aggressive control character cleaning for JSON strings
        # Replace newlines and tabs in JSON string values with spaces
        json_text = re.sub(r'[\r\n\t]', ' ', json_text)
        # Remove other control characters that can break JSON parsing
        json_text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', json_text)
        # Clean up multiple spaces
        json_text = re.sub(r'\s+', ' ', json_text)
        
        # Try to parse the JSON
        try: 
            loaded_json = json.loads(json_text)
            return loaded_json
        except json.JSONDecodeError as parse_error:
            # Debug information when JSON parsing fails
            print("Non-ASCII characters found:")
            for i, char in enumerate(json_text):
                if ord(char) > 127:
                    print(f"Position {i}: {repr(char)} (Unicode: U+{ord(char):04X})")
            
            print(f"JSON parse error at line {parse_error.lineno}, column {parse_error.colno}")
            print(f"Error message: {parse_error.msg}")
            
            # Show the problematic area
            lines = json_text.split('\n')
            if parse_error.lineno <= len(lines):
                problematic_line = lines[parse_error.lineno - 1]
                print(f"Problematic line: {repr(problematic_line)}")
            
            # Re-raise the error so it's caught by the outer except block
            raise parse_error
            
    except json.JSONDecodeError as e:
        self.logger.error(f"❌ Invalid JSON in response: {e}")
        self.logger.error(f"Response: {content[:200]}...")  # Show first 200 chars
        return {}
    except Exception as e:
        self.logger.warning(f"Failed to parse JSON response: {e}")
        return {}

# Remove this line - it should be called from within your class instance
_parse_json_response(raw_response)


```json
{
  "relationships": {
    "Alice": {
      "label": "Cautious Observer",
      "trust_delta": -1,
      "affection_delta": 0,
      "beat_memory": "Her persistent unease, while frustrating, highlighted a crucial difference in our approaches to problem-solving."
    },
    "Bob": {
      "label": "Pragmatic Skeptic",
      "trust_delta": -2,
      "affection_delta": -1,
      "beat_memory": "His dismissive attitude towards her concerns reinforced my initial feeling that he prioritized practical solutions over deeper investigation.”
    }
  },
  "knowledge_gained": {
    "facts": [
      "The storage room was forcibly entered.",
      "Someone stole our food.",
      "Bob is focused on securing the physical evidence and minimizing potential 'fantastical scenarios'."
    ],
    "suspicions": [
      "Bob's insistence on dismissing Alice's concerns suggests he might be deliberately overlooking alternative explanations.",
      "The forced entry indicates a deliberate act, potenti

{'relationships': {'Alice': {'label': 'Cautious Observer',
   'trust_delta': -1,
   'affection_delta': 0,
   'beat_memory': 'Her persistent unease, while frustrating, highlighted a crucial difference in our approaches to problem-solving.'},
  'Bob': {'label': 'Pragmatic Skeptic',
   'trust_delta': -2,
   'affection_delta': -1,
   'beat_memory': 'His dismissive attitude towards her concerns reinforced my initial feeling that he prioritized practical solutions over deeper investigation.'}},
 'knowledge_gained': {'facts': ['The storage room was forcibly entered.',
   'Someone stole our food.',
   "Bob is focused on securing the physical evidence and minimizing potential 'fantastical scenarios'."],
  'suspicions': ["Bob's insistence on dismissing Alice's concerns suggests he might be deliberately overlooking alternative explanations.",
   'The forced entry indicates a deliberate act, potentially involving someone who knew about our food supply.'],
  'gossip_worthy': ["Bob's aggressive exam

In [ ]:
# Parse the JSON
parsed_result = llm._parse_json_response(raw_response)

NameError: name 'llm' is not defined

## Test Relationship Updates

Once JSON parsing works, test updating relationships in the NPC manager.

In [ ]:
# Choose which character is doing the reflection
reflecting_character = "Charlie"  # Change this to Alice, Bob, or Charlie

print(f"=== Testing Relationship Update for {reflecting_character} ===")

# Load the current NPC manager
npc_manager = NPCManager.load_from_file("saves/npc_data.json", ["Alice", "Bob", "Charlie"])

# Get current relationships
print("\n--- Before Update ---")
for other_char in ["Alice", "Bob", "Charlie"]:
    if other_char != reflecting_character:
        rel = npc_manager.relationships.get_relationship(reflecting_character, other_char)
        if rel:
            print(f"{reflecting_character} -> {other_char}: trust={rel.trust}, affection={rel.affection}, status={rel.status.value}")

# Process the reflection if parsing was successful
if parsed_result:
    print("\n--- Processing Reflection ---")
    try:
        npc_manager.process_beat_reflection(reflecting_character, parsed_result)
        print("✅ Reflection processed successfully!")
        
        # Check updated relationships
        print("\n--- After Update ---")
        for other_char in ["Alice", "Bob", "Charlie"]:
            if other_char != reflecting_character:
                rel = npc_manager.relationships.get_relationship(reflecting_character, other_char)
                if rel:
                    print(f"{reflecting_character} -> {other_char}: trust={rel.trust}, affection={rel.affection}, status={rel.status.value}")
                    if rel.last_interaction:
                        print(f"  Last interaction: {rel.last_interaction[:50]}...")
        
        # Save the results
        npc_manager.save_to_file("saves/npc_data.json")
        print("\n✅ Results saved to npc_data.json")
        
    except Exception as e:
        print(f"❌ Error processing reflection: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n⚠️ Cannot process reflection - JSON parsing failed")

## Test Truncated JSON

Test how the system handles truncated/incomplete JSON responses.

In [ ]:
# Example of truncated JSON (like what we see in the logs)
truncated_response = '''```json
{
  "relationships": {
    "Alice": {
      "label": "Cautious Observer",
      "trust_delta": -1,
      "affection_delta": 0,
      "beat_memory": "Her persistent unease, while frustrating, h
'''

print("=== Testing Truncated JSON Handling ===")
print(f"Truncated response:\n{truncated_response}\n")

# Try to parse it
truncated_result = llm._parse_json_response(truncated_response)

if truncated_result:
    print("✅ Somehow parsed the truncated JSON!")
    print(f"Keys recovered: {list(truncated_result.keys())}")
    if 'relationships' in truncated_result:
        print(f"Relationships recovered: {list(truncated_result.get('relationships', {}).keys())}")
else:
    print("❌ Failed to parse truncated JSON (as expected)")
    print("\nThis is why reflections are failing - the LLM response gets cut off mid-JSON.")

## Test JSON Repair Function

Test the JSON repair function directly to see what it can recover.

In [ ]:
# Test the repair function on truncated JSON
test_cases = [
    # Case 1: Missing closing quotes and braces
    '''{\n      "relationships": {\n        "Alice": {\n          "label": "Test Label",\n          "trust_delta": 1,\n          "beat_memory": "This is a test that got cut off''',
    
    # Case 2: Complete but with control characters
    '{\n      "test": "value with\\nnewline and\\ttab"\n    }',
    
    # Case 3: Unicode quotes
    '{\n      "test": "value with fancy quotes"\n    }'
]

print("=== Testing JSON Repair Function ===")

for i, test_json in enumerate(test_cases, 1):
    print(f"\nTest Case {i}:")
    print(f"Input: {test_json[:50]}..." if len(test_json) > 50 else f"Input: {test_json}")
    
    # Try repair
    repaired = llm._attempt_json_repair(test_json)
    print(f"Repaired: {repaired[:50]}..." if len(repaired) > 50 else f"Repaired: {repaired}")
    
    # Try to parse
    try:
        result = json.loads(repaired)
        print("✅ Successfully parsed after repair!")
    except json.JSONDecodeError as e:
        print(f"❌ Still failed after repair: {e}")

## View Current Relationships

Quick view of all current relationships in the save file.

In [ ]:
# Load and display all current relationships
print("=== Current Relationships in Save File ===")

with open('saves/npc_data.json', 'r') as f:
    data = json.load(f)

relationships = data['relationships']['relationships']

for rel_key, rel_data in relationships.items():
    if rel_data['trust'] is not None:
        print(f"\n{rel_key}:")
        print(f"  Status: {rel_data['status']}")
        print(f"  Trust: {rel_data['trust']}")
        print(f"  Affection: {rel_data['affection']}")
        print(f"  Label: {rel_data.get('label', 'N/A')}")
        if rel_data.get('last_interaction'):
            print(f"  Last interaction: {rel_data['last_interaction'][:60]}...")
    else:
        print(f"\n{rel_key}: Not established (null values)")

# Count established relationships
established = sum(1 for r in relationships.values() if r['trust'] is not None)
print(f"\n📊 Summary: {established}/{len(relationships)} relationships established")